In [13]:
import pickle
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd
import numpy as np
import re

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

#response = requests.get('https://www.baseball-reference.com/players/g/gonzaad01.shtml')

#soup = BeautifulSoup(response.text, 'html5lib')

player_list = []
batting_stats = []
player_name = ''
player_salary = ''
team_list = []

def page_scrape(url):
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html5lib')
    
    return soup

def get_team(url):
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html5lib')
    
    url_origin = 'http://www.baseball-reference.com'
    
    team_urls = []
    
    standings_html = soup.find(string=re.compile('id="expanded_standings_overall"'))
    standings_soup = BeautifulSoup(standings_html, "html.parser")
    standings_table = standings_soup.find('tbody')
    
    links = standings_table.find_all('a')
    
    for link in links:
        team_urls.append(url_origin + link.get('href'))
    
    return team_urls


def get_player(url):
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html5lib')
    
    url_origin = 'http://www.baseball-reference.com'
    
    player_urls = []
    
    forty_man_html = soup.find(string=re.compile('id="the40man"'))
    forty_man_soup = BeautifulSoup(forty_man_html, "html.parser")
    forty_man_table = forty_man_soup.find('tbody')
    
    player_row = forty_man_table.find('td', {'data-row':"0"})
    
    links = forty_man_table.find_all('a')
    
    for link in links:
        player_urls.append(url_origin + link.get('href'))
    
    return player_urls

def get_2018_salary(soup, year):
    if soup.find('div', id="div_br-salaries") is not None:
        salary_table = soup.find('div', id="div_br-salaries")
        if salary_table.find('td', {'data-year':"2018"}) is not None:
            return salary_table.find('td', {'data-year':"2018"}).text


def get_full_name(soup):
    if soup.find('div', id='meta') is not None:
        player_info_table = soup.find('div', id='meta')
        return player_info_table.find('h1', itemprop="name").text


def get_batting_standard_table(soup):
    if soup.find('table', id='batting_standard') is not None:
        standard_batting_table = soup.find('table', id='batting_standard')
        return standard_batting_table
    else:
        return None


def get_batting_stats(soup):
    if soup is not None:
        table_body = soup.find('tbody')
        for row_year in table_body.find_all('tr', class_="full"):
            year_id = row_year.find('th').text
            actual_info = [e.text for e in row_year.find_all('td')]
            actual_info.insert(0, year_id)
            actual_info.insert(0, player_full_name)
            actual_info.insert(-1, player_dWAR)
            actual_info.insert(-1, player_salary)
            batting_stats.append(actual_info)
    
        return batting_stats
    else:
        pass

def get_player_dWAR(soup):
    player_dwars = []
    if soup.find('table', id='batting_value') is not None:
        batting_value_table = soup.find('table', id='batting_value')
        if batting_value_table.find('tr', id='batting_value.2011') is not None:
            year_2011 = batting_value_table.find('tr', id='batting_value.2011')
            if year_2011.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2011.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2012') is not None:
            year_2012 = batting_value_table.find('tr', id='batting_value.2012')
            if year_2012.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2012.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2013') is not None:
            year_2013 = batting_value_table.find('tr', id='batting_value.2013')
            if year_2013.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2013.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2014') is not None:
            year_2014 = batting_value_table.find('tr', id='batting_value.2014')
            if year_2014.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2014.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2015') is not None:
            year_2015 = batting_value_table.find('tr', id='batting_value.2015')
            if year_2015.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2015.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2016') is not None:
            year_2016 = batting_value_table.find('tr', id='batting_value.2016')
            if year_2016.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2016.find('td', {'data-stat':"WAR_def"}).text)
        if batting_value_table.find('tr', id='batting_value.2017') is not None:
            year_2017 = batting_value_table.find('tr', id='batting_value.2017')
            if year_2017.find('td', {'data-stat':"WAR_def"}).text != '':
                player_dwars.append(year_2017.find('td', {'data-stat':"WAR_def"}).text)       
    
    
    player_dwars = [float(i) for i in player_dwars]
    player_dWAR = float(sum(player_dwars)) / max(len(player_dwars), 1)
    return player_dWAR 
    
            

team_list = ['https://www.baseball-reference.com/teams/KCR/2018.shtml',
             'https://www.baseball-reference.com/teams/BAL/2018.shtml',
             'https://www.baseball-reference.com/teams/BOS/2018.shtml',
             'https://www.baseball-reference.com/teams/NYY/2018.shtml',
             'https://www.baseball-reference.com/teams/HOU/2018.shtml',
             'https://www.baseball-reference.com/teams/SEA/2018.shtml',
             'https://www.baseball-reference.com/teams/CHC/2018.shtml',
             'https://www.baseball-reference.com/teams/MIL/2018.shtml',
             'https://www.baseball-reference.com/teams/PHI/2018.shtml',
             'https://www.baseball-reference.com/teams/OAK/2018.shtml',
             'https://www.baseball-reference.com/teams/ATL/2018.shtml',
             'https://www.baseball-reference.com/teams/CLE/2018.shtml',
             'https://www.baseball-reference.com/teams/ARI/2018.shtml']

still_to_do = ['https://www.baseball-reference.com/teams/CHW/2018.shtml']

done_list = ['https://www.baseball-reference.com/teams/MIN/2018.shtml',
             'https://www.baseball-reference.com/teams/CIN/2018.shtml',
             'https://www.baseball-reference.com/teams/TEX/2018.shtml',
             'https://www.baseball-reference.com/teams/DET/2018.shtml',
             'https://www.baseball-reference.com/teams/SDP/2018.shtml',
             'https://www.baseball-reference.com/teams/NYM/2018.shtml',
             'https://www.baseball-reference.com/teams/MIA/2018.shtml',
             'https://www.baseball-reference.com/teams/CHW/2018.shtml']
        
#for team in get_team('https://www.baseball-reference.com/leagues/MLB-standings.shtml'):
#for player in get_player(team):
#        player_list.append(player)

for team in team_list:
    for player in get_player(team):
        player_list.append(player)

    
for player in player_list:
    player_soup = page_scrape(player)
    player_full_name = get_full_name(player_soup)
    player_dWAR = get_player_dWAR(player_soup)
    player_salary = get_2018_salary(player_soup, 2018)
    standard_batting = get_batting_standard_table(player_soup)
    get_batting_stats(standard_batting)
    time.sleep(1)

In [14]:
player_full_name

'Christian Walker'

In [15]:
big_group_top40 = pd.DataFrame(batting_stats, columns=['Player_Name','Year','Age','Team','League',
                                                        'Games_Played','Plate_Appearances','At_Bats',
                                                        'Runs','Hits','2B','3B','HR','RBI','SB','CS',
                                                        'BB','SO','BA','OBP','SLG','OPS','OPS+','TB',
                                                        'GDP','HBP','SH','SF','IBB','Pos','dWAR','Salary', 'Awards'])

In [16]:
big_group_top40

,Player_Name,Year,Age,Team,League,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,...,TB,GDP,HBP,SH,SF,IBB,Pos,dWAR,Salary,Awards
0,Jason Adam,2018,26,KCR,AL,1,0,0,0,0,...,0,0,0,0,0,0,1,0.000000,None,
1,Jason Hammel,2007,24,TBD,AL,1,1,1,0,1,...,2,0,0,0,0,0,1,0.000000,"$2,000,000",
2,Jason Hammel,2008,25,TBR,AL,2,0,0,0,0,...,0,0,0,0,0,0,1,0.000000,"$2,000,000",
3,Jason Hammel,2009,26,COL,NL,32,60,55,1,6,...,7,2,0,3,0,0,1,0.000000,"$2,000,000",
4,Jason Hammel,2010,27,COL,NL,29,63,57,3,6,...,7,0,0,5,0,0,1,0.000000,"$2,000,000",
5,Jason Hammel,2011,28,COL,NL,32,61,46,3,7,...,11,0,0,14,0,0,1,0.000000,"$2,000,000",
6,Jason Hammel,2012,29,BAL,AL,2,7,4,0,0,...,0,0,0,3,0,0,1,0.000000,"$2,000,000",
7,Jason Hammel,2013,30,BAL,AL,1,4,3,0,0,...,0,0,0,1,0,0,1,0.000000,"$2,000,000",
8,Jason Hammel,2014,31,TOT,MLB,19,43,37,2,5,...,5,1,0,5,0,0,1,0.000000,"$2,000,000",
9,Jason Hammel,2015,32,CHC,NL,31,70,65,6,11,...,12,1,0,3,1,0,1,0.000000,"$2,000,000",


In [17]:
import pickle

with open('big_group_top40.pkl', 'wb') as picklefile:
    pickle.dump(big_group_top40, picklefile)